In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung.csv?fbclid=IwAR1eS1GDDvk2XHzLVB_VyO_SnydiuFcdkIo_P3D0kg6j5LvAjXpYGbEOHQo', header = None)
df_clean = df
y = df_clean[0].values                       ## แยก X,y จาก Class
X = df_clean.drop([0],axis = 1).values

In [3]:
from imblearn.over_sampling import SMOTE   ## ใช้เมื่อ Class ไม่เท่ากัน
sm = SMOTE(random_state=1)
X, y =sm.fit_resample(X,y)

In [4]:
from sklearn.model_selection import train_test_split                                    # Separate Data into Train & Test (Note: Imbalance Dataset)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [5]:
from sklearn.preprocessing import MinMaxScaler    
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

# SVM

In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {
    'C' : [1, 2, 4, 8, 16, 32], # High C = Overfitting
    'gamma' : [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32] # High gamma = Overfitting
}
clf = GridSearchCV(SVC(),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

Best params : {'C': 32, 'gamma': 0.03125}
10CV accuracy : 74.81132075471699


In [7]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)
target_names = ['negative', 'positive']
sum(y_test == y_predict)/len(y_test)*100

79.54545454545455

# Blind test

In [8]:
df_test = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung_test.csv?fbclid=IwAR1CpkdjzdhmjGxNetyDpgUrxBYkHDj8GSpRTyxnNqpXmsJvyerWYR4VCpU', header = None)
df_test

,0,1,2,3,4,5,6,7,8,9,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,1,-0.979046,-0.716633,-0.981584,-0.961070,-0.964285,-0.962839,-0.849486,-0.900600,-0.732778,...,-0.007953,-0.835510,-0.991348,-0.860071,-0.925373,-1.000000,-1.000000,-0.581632,-0.841275,-0.997440
1,1,-0.887534,-0.057599,-0.963168,-0.941605,-0.958333,-0.915544,-0.473192,-0.694227,-0.081178,...,0.137629,-0.508157,-0.921880,-0.315089,0.074620,-0.897059,-0.949368,-0.727701,-0.545253,-0.966538
2,1,-0.976713,-0.802425,-0.970534,-0.985401,-0.988095,-0.961150,-0.831243,-0.907733,-0.740678,...,0.281941,-0.770071,-0.989058,-0.701209,-0.932835,-1.000000,-0.996383,-0.749610,-0.756818,-0.993245
3,1,-0.961500,-0.645135,-0.944753,-0.902674,-0.970238,-0.917232,-0.753708,-0.845328,-0.591923,...,0.445740,-0.560465,-0.972264,-0.674904,-0.305949,-0.872551,-0.949368,-0.640058,-0.580938,-0.991529
4,1,-0.958373,-0.249974,-0.845303,-0.892944,-0.904763,-0.903720,-0.377416,-0.864943,-0.286527,...,0.357187,-0.798273,-0.983715,-0.585479,-0.365677,-0.833334,-0.887884,-0.826811,-0.819876,-0.996596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,0,-0.945586,0.230922,-0.454894,-0.489033,-0.607139,-0.505091,-0.069551,-0.778469,0.136026,...,-0.422793,-0.180304,-0.929768,-0.335076,1.089560,-0.049037,-0.385160,-0.342718,-0.081677,-0.855971
268,0,-0.942505,-0.586638,-0.790055,-0.742090,-0.648807,-0.826015,-0.231462,-0.941610,-0.523474,...,0.106617,-0.823535,-0.990712,-0.900055,-0.813437,-0.936275,-0.898733,-0.586853,-0.836350,-0.998654
269,0,-0.988380,-0.857016,-0.922653,-0.863746,-0.875001,-0.932435,-0.559865,-0.957211,-0.820971,...,0.453743,-0.800414,-0.995038,-0.936874,-0.738810,-0.926472,-0.945752,-0.744389,-0.826324,-0.998681
270,0,-0.972186,-0.915509,-0.896870,-0.888077,-0.821427,-0.934124,-0.765102,-0.987074,-0.905221,...,0.660038,-0.796146,-0.995165,-0.956864,-0.888063,-0.975490,-0.938516,-0.907145,-0.793863,-0.996860


In [9]:
yt = df_clean[0].values                       ## แยก X,y จาก Class
Xt = df_clean.drop([0],axis = 1).values

In [10]:
X_test_norm = scaler.transform(Xt)

In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)
target_names = ['negative', 'positive']
sum(yt == y_predict)/len(yt)*100

88.92405063291139